# Tutorial 3: Optimal Weighting (Two-Step Estimation)

This tutorial explains **optimal weighting** in GMM/SMM and demonstrates the **two-step estimation** procedure.

## What You'll Learn

1. Why the weighting matrix matters
2. Identity vs optimal weighting
3. The two-step estimation procedure
4. Efficiency gains from optimal weighting

## Prerequisites

- Completed Tutorials 1-2
- Basic understanding of variance and efficiency

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from momentest import (
    gmm_estimate,
    smm_estimate,
    linear_iv,
    table_estimates,
    confidence_interval,
    load_econ381,
)

np.random.seed(42)

## 1. The GMM/SMM Objective Function

Recall the GMM/SMM objective:

$$Q(\theta) = \bar{g}(\theta)' W \bar{g}(\theta)$$

where:
- $\bar{g}(\theta)$ = sample/simulated moments (k × 1)
- $W$ = weighting matrix (k × k)

### Why Does W Matter?

The weighting matrix determines:
1. **Which moments get more weight** in the objective
2. **The efficiency** of the estimator (variance of $\hat{\theta}$)

Different W → Same point estimate (asymptotically) but different standard errors!

## 2. Identity Weighting

The simplest choice is $W = I$ (identity matrix):

$$Q(\theta) = \bar{g}(\theta)' I \bar{g}(\theta) = \sum_{j=1}^k \bar{g}_j(\theta)^2$$

This treats all moments equally. It's:
- **Simple**: No need to estimate W
- **Consistent**: Still gives correct point estimates
- **Inefficient**: Doesn't account for moment correlations

## 3. Optimal Weighting

Hansen (1982) showed that the **efficient** choice is:

$$W^* = S^{-1}$$

where $S = Var(\bar{g}(\theta_0))$ is the covariance matrix of the moment conditions.

### Intuition

- Moments with **high variance** get **less weight**
- Moments with **low variance** get **more weight**
- Correlated moments are properly accounted for

This is like **GLS** (Generalized Least Squares) for moment conditions!

## 4. The Two-Step Procedure

The problem: We need $S$ to compute $W^*$, but $S$ depends on $\theta_0$ (unknown).

**Solution: Two-step estimation**

1. **Step 1**: Estimate with $W = I$ → get $\hat{\theta}_1$
2. **Compute**: $\hat{S} = \hat{Var}(\bar{g}(\hat{\theta}_1))$
3. **Step 2**: Re-estimate with $W = \hat{S}^{-1}$ → get $\hat{\theta}_2$ (efficient)

This is what `weighting="optimal"` does automatically!

## 5. Example: Linear IV Model

Let's compare identity vs optimal weighting:

In [ ]:
# Generate data
dgp = linear_iv(n=500, seed=42, beta0=1.0, beta1=2.0, rho=0.5)

# Add extra instruments for overidentification
dgp.data['Z2'] = dgp.data['Z']**2
dgp.data['Z3'] = dgp.data['Z']**3

def moment_func(data, theta):
    """4 moments, 2 parameters (overidentified)."""
    beta0, beta1 = theta
    residual = data['Y'] - beta0 - beta1 * data['X']
    
    return np.column_stack([
        residual,
        residual * data['Z'],
        residual * data['Z2'],
        residual * data['Z3'],
    ])

print(f"True parameters: β₀={dgp.true_theta[0]}, β₁={dgp.true_theta[1]}")
print(f"Model: k=4 moments, p=2 parameters (overidentified)")

In [ ]:
# Identity weighting (one-step)
result_identity = gmm_estimate(
    data=dgp.data,
    moment_func=moment_func,
    bounds=[(-10, 10), (-10, 10)],
    k=4,
    weighting="identity",
)

# Optimal weighting (two-step)
result_optimal = gmm_estimate(
    data=dgp.data,
    moment_func=moment_func,
    bounds=[(-10, 10), (-10, 10)],
    k=4,
    weighting="optimal",
)

print("\n" + "="*70)
print("COMPARISON: Identity vs Optimal Weighting")
print("="*70)
print(f"{'Weighting':<15} {'β₀':>10} {'SE(β₀)':>10} {'β₁':>10} {'SE(β₁)':>10}")
print("-"*55)
print(f"{'Identity':<15} {result_identity.theta[0]:>10.4f} {result_identity.se[0]:>10.4f} "
      f"{result_identity.theta[1]:>10.4f} {result_identity.se[1]:>10.4f}")
print(f"{'Optimal':<15} {result_optimal.theta[0]:>10.4f} {result_optimal.se[0]:>10.4f} "
      f"{result_optimal.theta[1]:>10.4f} {result_optimal.se[1]:>10.4f}")
print(f"{'True':<15} {dgp.true_theta[0]:>10.4f} {'-':>10} {dgp.true_theta[1]:>10.4f} {'-':>10}")
print("="*70)

In [ ]:
# Efficiency gain
se_ratio_b0 = result_identity.se[0] / result_optimal.se[0]
se_ratio_b1 = result_identity.se[1] / result_optimal.se[1]

print(f"\nEfficiency gain (SE ratio):")
print(f"  β₀: {se_ratio_b0:.2f}x (optimal is {se_ratio_b0:.0%} as efficient)")
print(f"  β₁: {se_ratio_b1:.2f}x (optimal is {se_ratio_b1:.0%} as efficient)")
print(f"\nOptimal weighting gives LOWER standard errors!")

## 6. Monte Carlo: Efficiency Comparison

Let's run a Monte Carlo simulation to see the efficiency gain more clearly:

In [ ]:
# Monte Carlo simulation
n_mc = 100
estimates_identity = []
estimates_optimal = []

print("Running Monte Carlo simulation...")
for i in range(n_mc):
    # Generate new data
    dgp_mc = linear_iv(n=500, seed=i, beta0=1.0, beta1=2.0, rho=0.5)
    dgp_mc.data['Z2'] = dgp_mc.data['Z']**2
    dgp_mc.data['Z3'] = dgp_mc.data['Z']**3
    
    # Identity weighting
    try:
        res_id = gmm_estimate(
            data=dgp_mc.data, moment_func=moment_func,
            bounds=[(-10, 10), (-10, 10)], k=4, weighting="identity",
        )
        estimates_identity.append(res_id.theta)
    except:
        pass
    
    # Optimal weighting
    try:
        res_opt = gmm_estimate(
            data=dgp_mc.data, moment_func=moment_func,
            bounds=[(-10, 10), (-10, 10)], k=4, weighting="optimal",
        )
        estimates_optimal.append(res_opt.theta)
    except:
        pass
    
    if (i + 1) % 20 == 0:
        print(f"  Completed {i+1}/{n_mc}")

estimates_identity = np.array(estimates_identity)
estimates_optimal = np.array(estimates_optimal)

print(f"\nCompleted {len(estimates_identity)} identity, {len(estimates_optimal)} optimal")

In [ ]:
# Compare distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for i, (ax, param_name, true_val) in enumerate(zip(axes, ['β₀', 'β₁'], [1.0, 2.0])):
    ax.hist(estimates_identity[:, i], bins=20, alpha=0.5, label='Identity', density=True)
    ax.hist(estimates_optimal[:, i], bins=20, alpha=0.5, label='Optimal', density=True)
    ax.axvline(true_val, color='red', linestyle='--', linewidth=2, label='True')
    ax.set_xlabel(param_name)
    ax.set_ylabel('Density')
    ax.set_title(f'Distribution of {param_name} Estimates')
    ax.legend()

plt.tight_layout()
plt.show()

# Summary statistics
print("\nMonte Carlo Results:")
print(f"{'Weighting':<12} {'Mean(β₁)':>10} {'Std(β₁)':>10} {'RMSE(β₁)':>10}")
print("-"*45)
bias_id = estimates_identity[:, 1].mean() - 2.0
std_id = estimates_identity[:, 1].std()
rmse_id = np.sqrt(bias_id**2 + std_id**2)
print(f"{'Identity':<12} {estimates_identity[:, 1].mean():>10.4f} {std_id:>10.4f} {rmse_id:>10.4f}")

bias_opt = estimates_optimal[:, 1].mean() - 2.0
std_opt = estimates_optimal[:, 1].std()
rmse_opt = np.sqrt(bias_opt**2 + std_opt**2)
print(f"{'Optimal':<12} {estimates_optimal[:, 1].mean():>10.4f} {std_opt:>10.4f} {rmse_opt:>10.4f}")
print(f"{'True':<12} {2.0:>10.4f}")

print(f"\nEfficiency gain: Optimal has {std_id/std_opt:.1%} lower std dev")

## 7. When Does Optimal Weighting Help Most?

Optimal weighting helps most when:

1. **Moments have different variances**: High-variance moments get downweighted
2. **Moments are correlated**: Correlations are properly accounted for
3. **Model is overidentified**: More moments → more room for efficiency gains

### When It Doesn't Matter

- **Exactly identified** (k = p): All weighting matrices give the same estimate
- **Homoskedastic, uncorrelated moments**: Identity is already optimal

In [ ]:
# Exactly identified case (k = p = 2)
def moment_func_exact(data, theta):
    """2 moments, 2 parameters (exactly identified)."""
    beta0, beta1 = theta
    residual = data['Y'] - beta0 - beta1 * data['X']
    return np.column_stack([residual, residual * data['Z']])

dgp_exact = linear_iv(n=500, seed=42)

res_exact_id = gmm_estimate(
    data=dgp_exact.data, moment_func=moment_func_exact,
    bounds=[(-10, 10), (-10, 10)], k=2, weighting="identity",
)

res_exact_opt = gmm_estimate(
    data=dgp_exact.data, moment_func=moment_func_exact,
    bounds=[(-10, 10), (-10, 10)], k=2, weighting="optimal",
)

print("Exactly Identified (k=p=2):")
print(f"  Identity: β₁ = {res_exact_id.theta[1]:.4f}")
print(f"  Optimal:  β₁ = {res_exact_opt.theta[1]:.4f}")
print(f"\nSame estimate! Weighting doesn't matter when exactly identified.")

## 8. The Weighting Matrix in Detail

Let's look at what the optimal weighting matrix looks like:

In [ ]:
# Get the moment covariance at the estimate
from momentest import GMMEngine

engine = GMMEngine(
    data=dgp.data,
    k=4,
    p=2,
    moment_func=moment_func,
)

_, S = engine.moments(result_optimal.theta)

print("Moment Covariance Matrix S:")
print(np.round(S, 4))

print("\nOptimal Weighting Matrix W = S⁻¹:")
W_opt = np.linalg.inv(S)
print(np.round(W_opt, 4))

In [ ]:
# Visualize the weighting matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Covariance matrix
im1 = axes[0].imshow(S, cmap='RdBu_r', aspect='auto')
axes[0].set_title('Moment Covariance S')
axes[0].set_xticks(range(4))
axes[0].set_yticks(range(4))
axes[0].set_xticklabels(['g₁', 'g₂', 'g₃', 'g₄'])
axes[0].set_yticklabels(['g₁', 'g₂', 'g₃', 'g₄'])
plt.colorbar(im1, ax=axes[0])

# Weighting matrix
im2 = axes[1].imshow(W_opt, cmap='RdBu_r', aspect='auto')
axes[1].set_title('Optimal Weighting W = S⁻¹')
axes[1].set_xticks(range(4))
axes[1].set_yticks(range(4))
axes[1].set_xticklabels(['g₁', 'g₂', 'g₃', 'g₄'])
axes[1].set_yticklabels(['g₁', 'g₂', 'g₃', 'g₄'])
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

## 9. Exercises

### Exercise 1: Vary Overidentification
Compare efficiency gains with k=3, k=4, k=5 moments. Does more overidentification help?

### Exercise 2: Heteroskedasticity
Generate data with heteroskedastic errors. How much does optimal weighting help?

### Exercise 3: SMM Weighting
Repeat the comparison for SMM (Tutorial 2 example). Is the efficiency gain similar?

In [ ]:
# Exercise 1 starter code
print("Efficiency gain by number of moments:")
print(f"{'k':>5} {'SE(β₁) Identity':>18} {'SE(β₁) Optimal':>18} {'Ratio':>10}")
print("-"*55)

for k in [2, 3, 4]:
    # Define moment function with k moments
    def make_moment_func(k):
        def mf(data, theta):
            beta0, beta1 = theta
            residual = data['Y'] - beta0 - beta1 * data['X']
            moments = [residual]
            if k >= 2:
                moments.append(residual * data['Z'])
            if k >= 3:
                moments.append(residual * data['Z']**2)
            if k >= 4:
                moments.append(residual * data['Z']**3)
            return np.column_stack(moments)
        return mf
    
    mf = make_moment_func(k)
    
    res_id = gmm_estimate(data=dgp.data, moment_func=mf, bounds=[(-10, 10), (-10, 10)], k=k, weighting="identity")
    res_opt = gmm_estimate(data=dgp.data, moment_func=mf, bounds=[(-10, 10), (-10, 10)], k=k, weighting="optimal")
    
    ratio = res_id.se[1] / res_opt.se[1] if res_opt.se[1] > 0 else np.nan
    print(f"{k:>5} {res_id.se[1]:>18.4f} {res_opt.se[1]:>18.4f} {ratio:>10.2f}")

## Summary

In this tutorial, you learned:

1. **Weighting matrix**: Determines which moments get more weight
2. **Identity weighting**: Simple but inefficient
3. **Optimal weighting**: $W = S^{-1}$ is efficient (Hansen 1982)
4. **Two-step procedure**: First estimate with identity, then re-estimate with optimal
5. **Efficiency gains**: Optimal weighting reduces standard errors

### Key Takeaways

- Use `weighting="optimal"` for efficient estimation
- Efficiency gains are larger with more overidentification
- For exactly identified models, weighting doesn't matter
- `momentest` handles two-step automatically

### Next Steps

- **Tutorial 4**: Bootstrap inference
- **Tutorial 5**: Diagnostics and visualization